In [3]:
!pip install pytorch_lightning
!pip install pytorch-lifestream

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 35.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.5 MB/s eta 0:00:00
  Created wheel for pytorch-lifestream: filename=pytorch_lifestream-0.5.2-py3-none-any.whl size=256963 sha256=8dad8fbce6dbdfca602b4e19c446cdf74b761d6d1d58b62c693b6ba4b15fe778
  Stored in directory: 

In [7]:
import pandas as pd
import os
import torch
import pytorch_lightning as pl

2023-03-24 00:13:14.460947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
!ls

In [18]:
import os

if not os.path.exists('data/transactions.csv'):
#    ! mkdir -p data
#    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip
#    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/0554f0cf/clickstream.zip
    ! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/e756bf99/train.csv
 #   ! unzip clickstream.zip -d data
  #  ! unzip transactions.zip 
 #   ! cp train.csv data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: self signed certificate in certificate chain
More details here: https://curl.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [20]:
! curl -OL https://storage.yandexcloud.net/datasouls-ods/materials/e756bf99/train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: self signed certificate in certificate chain
More details here: https://curl.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [ ]:
mkdir data

curl -OLhttps://storage.yandexcloud.net/datasouls-ods/materials/0433a4ca/transactions.zip
unzip transactions.zip -d data
mv transactions.zip data/

In [9]:
transactions = pd.read_csv('transactions.csv')

In [10]:
transactions.user_id.nunique()

22533

In [5]:
transactions.transaction_dttm = pd.to_datetime(transactions.transaction_dttm)

In [6]:
from ptls.preprocessing import PandasDataPreprocessor

In [7]:
preprocessor = PandasDataPreprocessor(
    col_id='user_id',
    col_event_time='transaction_dttm',
    event_time_transformation='dt_to_timestamp',
    cols_category=['mcc_code', 'currency_rk'],
    cols_numerical=['transaction_amt'],
    return_records=True,
)

In [8]:
%%time 
dataset = preprocessor.fit_transform(transactions)

/opt/conda/lib/python3.7/site-packages/ptls/preprocessing/pandas/event_time.py:8: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  return pd.to_datetime(x).astype('datetime64[s]').astype('int64') // 1000000000


CPU times: user 1min 26s, sys: 10.9 s, total: 1min 37s
Wall time: 1min 37s


In [9]:
import pickle

with open('preprocessor.p', 'wb') as f:
    pickle.dump(preprocessor, f)

In [10]:
dataset = sorted(dataset, key = lambda x: x['user_id'])

In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

len(train), len(test)

(18026, 4507)

Embedding training

In [12]:
transactions.mcc_code.nunique(), transactions.currency_rk.nunique(), transactions.transaction_dttm.nunique()

(386, 4, 12787367)

In [13]:
from functools import partial
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule

trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'transaction_amt': 'identity'},
    embeddings={
        'event_time': {'in': 800, 'out': 16},
        'mcc_code': {'in': 450, 'out': 16},
        'currency_rk':{'in': 4, 'out': 2}
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9),
)

In [14]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule

train_dl = PtlsDataModule(
    train_data=ColesDataset(
        MemoryMapDataset(
            data=train,
            i_filters=[
                SeqLenFilter(min_seq_len=25),
            ],
        ),
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200,
        ),
    ),
    train_num_workers=2,
    train_batch_size=256,
)

In [15]:


import logging

trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else 0,
    enable_progress_bar=False,
)

In [16]:
%%time
print(f'logger.version = {trainer.logger.version}')
trainer.fit(model, train_dl)
print(trainer.logged_metrics)

logger.version = 0


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")


{'loss': tensor(138.5396), 'seq_len': tensor(116.5208)}
CPU times: user 4min 8s, sys: 15.9 s, total: 4min 24s
Wall time: 5min 2s


In [17]:
torch.save(seq_encoder.state_dict(), "coles_emb.pt")

In [18]:
from ptls.data_load.datasets import inference_data_loader

train_dl = inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = torch.vstack(trainer.predict(model, train_dl, ))

test_dl = inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = torch.vstack(trainer.predict(model, test_dl))

train_embeds.shape, test_embeds.shape

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:175: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")


(torch.Size([18026, 256]), torch.Size([4507, 256]))

Готово!

In [19]:
df_target = pd.read_csv( 'data/train.csv')
df_target.rename(columns={'bank':'user_id'},inplace=True)
df_target = df_target.set_index('user_id')

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['user_id'] = [x['user_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='user_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['user_id'] = [x['user_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='user_id')

print(train_df.shape, test_df.shape)

(18026, 258) (4507, 258)


In [20]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
print(train_df.shape, test_df.shape)

(6838, 258) (1671, 258)


In [55]:
testdf= test_df.copy()
traindf = train_df.copy()

In [56]:
for column in testdf.columns:
    if type(testdf.column) != str:
        testdf[column] = testdf[column].apply(lambda x: x.item())

AttributeError: 'str' object has no attribute 'item'

In [58]:
testdf.to_csv('train_df.csv')

In [59]:
from IPython.display import FileLink
FileLink('train_df.csv')

/kaggle/working/train_df.csv

In [6]:
test_df.shape, train_df.shape

((1671, 259), (6838, 259))

# Learning COLES

In [4]:
import pandas as pd
import numpy as np

In [5]:
test_df = pd.read_csv('./test_df.csv')
train_df = pd.read_csv('./train_df.csv')

In [54]:
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV


In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [74]:
from sklearn.ensemble import RandomForestClassifier

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['higher_education']
x_test, y_test = test_df[embed_columns], test_df['higher_education']



In [90]:
#without resampling and gridsearch
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
y_proba= clf.predict_proba(x_test)
y_pred = clf.predict(x_test)
print(f'''accuracy: {clf.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)}, 
      roc_auc: {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7660083782166367 
      f1: 0.8531731130304168, 
      precision: 0.9161290322580645, 
      roc_auc: 0.7550267569792679


In [91]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
ros = RandomOverSampler(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0.0, 4923), (1.0, 4923)]


In [93]:
#gridsearch with randomoversampler
# params = {'n_estimators':[40, 70, 100, 150],
#          'max_depth':[ 3, 5, 7],
#          'max_features':[20, 40, 60, 80]}
params = {'n_estimators':[150, 170, 200],
         'max_depth':[7, 15, 20],
         'max_features':[20, 40, 60, 80, 150]}
clf = RandomForestClassifier()
gs = GridSearchCV(clf, params, scoring = 'roc_auc', cv = 3, verbose = 3, n_jobs =-1)
gs.fit(x_resampled, y_resampled)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [7, 15, 20],
                         'max_features': [20, 40, 60, 80, 150],
                         'n_estimators': [150, 170, 200]},
             scoring='roc_auc', verbose=3)

In [94]:
clf = gs.best_estimator_
clf.fit(x_resampled, y_resampled)
y_proba= clf.predict_proba(x_test)
y_pred = clf.predict(x_test)
print(f'''accuracy: {clf.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)},
      roc_auc: {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7624177139437462 
      f1: 0.8448612739351309, 
      precision: 0.8717741935483871,
      roc_auc: 0.758573460070354


In [95]:
ros = SMOTE(random_state=0)
x_resampled, y_resampled = ros.fit_resample(x_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0.0, 4923), (1.0, 4923)]


In [96]:
#smote without grid search
clf = RandomForestClassifier()
clf.fit(x_resampled, y_resampled)
y_proba= clf.predict_proba(x_test)
y_pred = clf.predict(x_test)
print(f'''accuracy: {clf.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)},
      roc_auc: {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7324955116696589 
      f1: 0.8170282439623414, 
      precision: 0.8048387096774193,
      roc_auc: 0.7498344061073273


In [97]:
#grid search with SMOTE
params = {'n_estimators':[150, 170, 200],
         'max_depth':[7, 15, 20],
         'max_features':[20, 40, 60, 80, 150]}
clf = RandomForestClassifier()
gs = GridSearchCV(clf, params, scoring = 'accuracy', cv = 3, verbose = 3, n_jobs =-1)
gs.fit(x_resampled, y_resampled)

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV 2/3] END max_depth=7, max_features=20, n_estimators=150;, score=0.858 total time=  12.1s
[CV 1/3] END max_depth=7, max_features=20, n_estimators=170;, score=0.858 total time=  14.6s
[CV 3/3] END max_depth=7, max_features=20, n_estimators=170;, score=0.866 total time=  13.9s
[CV 2/3] END max_depth=7, max_features=20, n_estimators=200;, score=0.860 total time=  16.9s
[CV 1/3] END max_depth=7, max_features=40, n_estimators=150;, score=0.860 total time=  23.1s
[CV 3/3] END max_depth=7, max_features=40, n_estimators=150;, score=0.873 total time=  23.8s
[CV 2/3] END max_depth=7, max_features=40, n_estimators=170;, score=0.865 total time=  27.3s
[CV 1/3] END max_depth=7, max_features=40, n_estimators=200;, score=0.863 total time=  31.0s
[CV 3/3] END max_depth=7, max_features=40, n_estimators=200;, score=0.873 total time=  31.3s
[CV 2/3] END max_depth=7, max_features=60, n_estimators=150;, score=0.869 total time=  35.0s
[CV 1/3]

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [7, 15, 20],
                         'max_features': [20, 40, 60, 80, 150],
                         'n_estimators': [150, 170, 200]},
             scoring='accuracy', verbose=3)

In [98]:
clf = gs.best_estimator_
clf.fit(x_resampled, y_resampled)
y_proba= clf.predict_proba(x_test)
y_pred = clf.predict(x_test)
print(f'''accuracy: {clf.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)},
      roc_auc: {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7348892878515859 
      f1: 0.8170177612556795, 
      precision: 0.7975806451612903,
      roc_auc: 0.7485648529301699


# With CatBoost embedding_features

In [16]:
from catboost import CatBoostClassifier, metrics

In [50]:
CatBoostModel_emb = CatBoostClassifier(
iterations= 500,
learning_rate = 0.05,
use_best_model = True,
eval_metric ='F1', 
#loss_function='Logloss',
random_seed = 42,
logging_level = 'Silent',
embedding_features = ['embeddings'],
depth = 5)

In [51]:
x_train, y_train = train_df[embed_columns], train_df['higher_education']
x_train_emb = pd.DataFrame(np.arange(len(x_train)))
x_train_emb['embeddings'] = x_train.values.tolist()
x_train_emb = x_train_emb.drop([0], axis = 1)
x_val, y_val = test_df[embed_columns], test_df['higher_education']
x_val_emb = pd.DataFrame(np.arange(len(x_val)))
x_val_emb['embeddings'] = x_val.values.tolist()
x_val_emb = x_val_emb.drop([0], axis = 1)

In [52]:
%%time

CatBoostModel_emb.fit(
    x_train_emb, y_train,
    eval_set=(x_val_emb, y_val),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CPU times: user 10.6 s, sys: 400 ms, total: 11 s
Wall time: 9.86 s


In [53]:
CatBoostModel_emb.get_best_score()

{'learn': {'Logloss': 0.5067433210222069, 'F1': 0.8451495920217589},
 'validation': {'Logloss': 0.5054655069813566, 'F1': 0.8585537918871252}}

# But without embedding_features seems to achieve better results 

In [57]:
x_train, y_train = train_df[embed_columns], train_df['higher_education']
x_test, y_test = test_df[embed_columns], test_df['higher_education']

In [58]:
CatBoostModel = CatBoostClassifier(
iterations= 500,
learning_rate = 0.05,
use_best_model = True,
eval_metric ='F1', 
loss_function='Logloss',
random_seed = 42,
logging_level = 'Silent',
depth = 5)

In [59]:
CatBoostModel.fit(
    x_train, y_train,
    eval_set=(x_test, y_test),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [36]:
CatBoostModel.get_best_score()

{'learn': {'Logloss': 0.3017351258348865, 'F1': 0.9396128286622365},
 'validation': {'Logloss': 0.4799109609095098, 'F1': 0.8614828209764918}}

In [60]:
y_pred = CatBoostModel.predict(x_test)
y_proba = CatBoostModel.predict_proba(x_test)

In [70]:
print(f'''accuracy: {CatBoostModel.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)}
     roc auc : {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7707959305804907 
      f1: 0.8614828209764918, 
      precision: 0.9604838709677419
     roc auc : 0.7583208592171244


# Learning Random encoder

In [32]:
import pickle
with open('random_embeddings.pickle', 'rb') as f:
    data = pickle.load(f)

In [33]:
data['user_id'] = transactions.user_id.unique()
train = pd.read_csv('train.csv')
data = data.merge(train, right_on = 'bank',left_on = 'user_id', how ='left')
data.drop(['bank', 'user_id'], inplace=True, axis = 1)
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

len(train), len(test)

(18026, 4507)

In [42]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train_df.shape, test_df.shape)

(6838, 259) (1671, 259)


In [45]:
embed_columns = [x for x in train.columns if x.startswith('emb')]
x_train, y_train = train[embed_columns], train['higher_education']
x_test, y_test = test[embed_columns], test['higher_education']

In [47]:
from catboost import CatBoostClassifier, metrics

/Users/eandrejchuk/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [48]:
CatBoostModel = CatBoostClassifier(
iterations= 500,
learning_rate = 0.05,
use_best_model = True,
eval_metric ='F1', 
loss_function='Logloss',
random_seed = 42,
logging_level = 'Silent',
depth = 5)

In [49]:
CatBoostModel.fit(
    x_train, y_train,
    eval_set=(x_test, y_test),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [50]:
CatBoostModel.get_best_score()

{'learn': {'Logloss': 0.43990702130215736, 'F1': 0.8686137827583164},
 'validation': {'Logloss': 0.5724590081328191, 'F1': 0.8519409137753349}}

In [51]:
y_pred = CatBoostModel.predict(x_test)
y_proba = CatBoostModel.predict_proba(x_test)

In [55]:
print(f'''accuracy: {CatBoostModel.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)}
     roc auc : {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7420706163973668 
      f1: 0.8519409137753349, 
      precision: 1.0
     roc auc : 0.5047021181049323


# Learning agg encoder

In [57]:
import pickle
with open('agg_feat_embed.pickle', 'rb') as f:
    data = pickle.load(f)

In [58]:
data['user_id'] = transactions.user_id.unique()
train = pd.read_csv('train.csv')
data = data.merge(train, right_on = 'bank',left_on = 'user_id', how ='left')
data.drop(['bank', 'user_id'], inplace=True, axis = 1)
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

len(train), len(test)

(18026, 4507)

In [59]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train_df.shape, test_df.shape)

(6838, 259) (1671, 259)


In [60]:
embed_columns = [x for x in train.columns if x.startswith('emb')]
x_train, y_train = train[embed_columns], train['higher_education']
x_test, y_test = test[embed_columns], test['higher_education']

In [61]:
CatBoostModel = CatBoostClassifier(
iterations= 500,
learning_rate = 0.05,
use_best_model = True,
eval_metric ='F1', 
loss_function='Logloss',
random_seed = 42,
logging_level = 'Silent',
depth = 5)

In [62]:
CatBoostModel.fit(
    x_train, y_train,
    eval_set=(x_test, y_test),
    plot=True
#     logging_level='Verbose',  # you can uncomment this for text output
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [63]:
y_pred = CatBoostModel.predict(x_test)
y_proba = CatBoostModel.predict_proba(x_test)

In [64]:
print(f'''accuracy: {CatBoostModel.score(x_test, y_test)} 
      f1: {f1_score(y_pred, y_test)}, 
      precision: {precision_score(y_pred, y_test)}
     roc auc : {roc_auc_score(y_test, y_proba[:,1])}''')

accuracy: 0.7426690604428486 
      f1: 0.852233676975945, 
      precision: 1.0
     roc auc : 0.5014276626001049
